In [1]:
import numpy as np 
import pandas as pd 
import torch
import torch.nn as nn
import torch.nn.functional as Func

import matplotlib.pyplot as plt

from tqdm import tqdm

In [2]:
torch.cuda.is_available()
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [3]:
path='./Iris.csv'
fullData=pd.read_csv(path)
fullData=fullData.sample(frac=1)
fullData[0:4]

# dataloader=torch.utils.data.DataLoader(dataset, batch_size=16,shuffle=True, num_workers = 4)

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
134,135,6.1,2.6,5.6,1.4,Iris-virginica
54,55,6.5,2.8,4.6,1.5,Iris-versicolor
2,3,4.7,3.2,1.3,0.2,Iris-setosa
12,13,4.8,3.0,1.4,0.1,Iris-setosa


In [4]:
fullData=fullData.drop(columns='Id')
clases=fullData.iloc[:,4].unique()
print(clases)

['Iris-virginica' 'Iris-versicolor' 'Iris-setosa']


In [5]:
newLabel={
    'Iris-setosa':0,
    'Iris-versicolor':1,
    'Iris-virginica':2
}
fullData.replace(newLabel,inplace=True)

In [6]:
fullData

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
134,6.1,2.6,5.6,1.4,2
54,6.5,2.8,4.6,1.5,1
2,4.7,3.2,1.3,0.2,0
12,4.8,3.0,1.4,0.1,0
123,6.3,2.7,4.9,1.8,2
...,...,...,...,...,...
92,5.8,2.6,4.0,1.2,1
136,6.3,3.4,5.6,2.4,2
140,6.7,3.1,5.6,2.4,2
99,5.7,2.8,4.1,1.3,1


In [7]:

CountOfRows=fullData.shape[0]
testData=fullData[0:CountOfRows//3]
trainData=fullData[CountOfRows//3:CountOfRows]



In [8]:
def normalizeCollumns(in_file):
    for i in range(in_file.shape[1]):
        column=in_file[in_file.columns[i]]
        in_file[in_file.columns[i]]=(column-np.min(column))/(np.max(column)-np.min(column))
    return in_file

In [9]:
class MyDataset(torch.utils.data.Dataset):
    def __init__(self,csv_file):
        data=csv_file
        x=data.iloc[:,:4]
        x=normalizeCollumns(x)
        y=data.iloc[:,4]
        self.x_train=torch.Tensor(np.array(x))
        self.y_train=nn.functional.one_hot(torch.Tensor(np.array(y)).to(torch.int64),num_classes=3)
        
        
    def __len__(self):
        return len(self.y_train)
    def __getitem__(self,idx):
        return self.x_train[idx],self.y_train[idx]

In [10]:
trainDataset=MyDataset(trainData)
trainDataloader= torch.utils.data.DataLoader(trainDataset,batch_size=5)

In [11]:
class IrisClassifier(torch.nn.Module):
    def __init__(self,countOfParametrs=4):
        super().__init__()
        act_func=torch.nn.ReLU()
        linear_1 = torch.nn.Linear(countOfParametrs,4*countOfParametrs)
        linear_2= torch.nn.Linear(4*countOfParametrs,3)

        self.model = torch.nn.Sequential(
            linear_1,act_func,linear_2
        )

    def forward(self,x):
        return self.model(x)

In [12]:
model=IrisClassifier().to(device)
loss_func=torch.nn.CrossEntropyLoss()
optimiser=torch.optim.Adam(model.parameters(),lr=1e-2)

In [13]:
def accruracy(pred,label):
    answer = pred.detach().numpy().argmax(1) == label.numpy().argmax(1) 
    return answer.mean()

In [14]:
def trainModel(dataloader,model,loss_fn,optimiser):
    loss_val=0
    accruracy_val=0
    for sample in tqdm(dataloader):
        X,Y=sample
        X=X.to(device)
        Y=Y.to(device)
        
        Y=Y.to(torch.float32)
        
        pred=model(X)
        loss=loss_fn(pred,Y)
        
        optimiser.zero_grad()
        loss.backward()
        optimiser.step()
        loss_val+=loss.item()
        accruracy_val+= accruracy(pred.cpu(),Y.cpu())
        
    print("avg loss=",loss_val/len(dataloader))
    print("avg accuracy=",accruracy_val/len(dataloader))
    


In [15]:
countOfEpoch=20
for epoch in range(countOfEpoch):
    print("Epoch#",epoch+1)
    trainModel(trainDataloader,model,loss_func,optimiser)

Epoch# 1


100%|██████████| 20/20 [00:02<00:00,  8.53it/s]


avg loss= 1.0280269920825957
avg accuracy= 0.55
Epoch# 2


100%|██████████| 20/20 [00:00<00:00, 636.81it/s]


avg loss= 0.7823014855384827
avg accuracy= 0.71
Epoch# 3


...

avg loss= 0.12931858794763684
avg accuracy= 0.9600000000000002
Epoch# 19


100%|██████████| 20/20 [00:00<00:00, 495.97it/s]


avg loss= 0.12490382557734847
avg accuracy= 0.9600000000000002
Epoch# 20


100%|██████████| 20/20 [00:00<00:00, 653.93it/s]

avg loss= 0.12100137928500772
avg accuracy= 0.9600000000000002


In [16]:

def testModel(dataloader,model):
    accruracy_val=0
    for sample in dataloader:
        X,Y=sample
        
        X=X.to(device)
        Y=Y.to(device)
        
        X=X.to(torch.float32)
        pred=model(X)
        accruracy_val+= accruracy(pred.cpu(),Y.cpu())
    print("accuracy=",accruracy_val/len(dataloader))

In [18]:
testDataset=MyDataset(testData)
testDataloader=torch.utils.data.DataLoader(testDataset,batch_size=1)
testModel(testDataloader,model)

accuracy= 0.98
